In [2]:
import os
import sys
import librosa
import tflearn
import tensorflow as tf
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
%matplotlib inline

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [3]:
os.chdir('C:/Users/karti/Speech/LibriSpeech/dev-clean/')

In [4]:
new_y=[]
f = open("y.txt", "r")
for x in f:
  new_y.append(int(x.strip()))

In [5]:
new_y = np.array(new_y).reshape(-1,1)

In [6]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(new_y)
res=enc.transform(new_y).toarray()

In [7]:
res

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [8]:
import pickle
new_mfccs_padded_mean=[]
with open('padding_mean_mfccs_flatten.pkl', 'rb') as f:
    new_mfccs_padded_mean = pickle.load(f)

In [9]:
len(new_mfccs_padded_mean)

2703

In [10]:
len(new_mfccs_padded_mean[0])

9542

In [11]:
new_mfccs_padded_mean[0]

[-522.7668,
 -515.66626,
 -513.8235,
 -519.6458,
 -529.6058,
 -541.97534,
 -549.1127,
 -549.66113,
 -552.7884,
 -563.20557,
 -579.0012,
 -586.83966,
 -589.7636,
 -593.21277,
 -596.7899,
 -599.3583,
 -599.8722,
 -599.43427,
 -601.7576,
 -604.087,
 -606.643,
 -604.26294,
 -594.7184,
 -526.10333,
 -404.20172,
 -306.72345,
 -255.19205,
 -213.24362,
 -215.07686,
 -296.11255,
 -315.65097,
 -200.70557,
 -166.29402,
 -219.2694,
 -347.09357,
 -369.06165,
 -234.85738,
 -180.19046,
 -183.31035,
 -187.5966,
 -167.54356,
 -178.9076,
 -190.67288,
 -211.18733,
 -214.15814,
 -238.41058,
 -201.85397,
 -163.89262,
 -180.3691,
 -236.24957,
 -270.1003,
 -284.22006,
 -300.93704,
 -292.44876,
 -239.6829,
 -210.28374,
 -206.7477,
 -198.26038,
 -196.3358,
 -232.9827,
 -284.94943,
 -268.66452,
 -243.05017,
 -214.7947,
 -191.89684,
 -183.97284,
 -172.51604,
 -189.80159,
 -269.9999,
 -454.94672,
 -532.72125,
 -285.48538,
 -223.09866,
 -229.87949,
 -240.87634,
 -227.84332,
 -241.51805,
 -210.14882,
 -177.11537,
 

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_mfccs_padded_mean, res, test_size=0.33, random_state=42)

In [13]:
net = tflearn.input_data(shape=[None, 9542]) #Two wave chunks
net = tflearn.fully_connected(net, 1024, activation='relu')
net = tflearn.dropout(net, 0.4)
net = tflearn.fully_connected(net, 512, activation='relu')
net = tflearn.dropout(net, 0.35)
net = tflearn.fully_connected(net, 256, activation='relu')
net = tflearn.dropout(net, 0.3)
net = tflearn.fully_connected(net, 128, activation='relu')
net = tflearn.dropout(net, 0.25)
net = tflearn.fully_connected(net, 64, activation='relu')
net = tflearn.dropout(net, 0.2)
net = tflearn.fully_connected(net, 40, activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
model = tflearn.DNN(net)

Instructions for updating:
Use tf.cast instead.


In [15]:
model.fit(X_train, y_train, n_epoch=250, show_metric=True, batch_size=128,snapshot_step=100)

Training Step: 3749  | total loss: 3.43022 | time: 0.747s
| Adam | epoch: 250 | loss: 3.43022 - acc: 0.0694 -- iter: 1792/1811
Training Step: 3750  | total loss: 3.42524 | time: 0.802s
| Adam | epoch: 250 | loss: 3.42524 - acc: 0.0695 -- iter: 1811/1811
--


In [16]:
y_pred = model.predict(X_train)

In [17]:
np.sum(np.argmax(y_train, axis=1)==np.argmax(y_pred, axis=1))/len(y_train)

0.03975704030922143

In [18]:
y_pred = model.predict(X_test)

In [19]:
np.sum(np.argmax(y_test, axis=1)==np.argmax(y_pred, axis=1))/len(y_test)

0.03139013452914798

In [20]:
model.save('model_padded_mean.tflearn')

INFO:tensorflow:C:\Users\karti\Speech\LibriSpeech\dev-clean\model_padded_mean.tflearn is not in all_model_checkpoint_paths. Manually adding it.


# Reload Saved Model

In [4]:
net = tflearn.input_data(shape=[None, 13]) #Two wave chunks
net = tflearn.fully_connected(net, 256, activation='relu')
net = tflearn.dropout(net, 0.4)
net = tflearn.fully_connected(net, 128, activation='relu')
net = tflearn.dropout(net, 0.25)
net = tflearn.fully_connected(net, 64, activation='relu')
net = tflearn.dropout(net, 0.2)
net = tflearn.fully_connected(net, 40, activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')
model = tflearn.DNN(net)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\karti\Speech\LibriSpeech\dev-clean\model_mean.tflearn


In [97]:
model.load('model_padded_mean.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\karti\Speech\LibriSpeech\dev-clean\model_mean.tflearn
